In [9]:
import numpy as np
import pandas as pd

In [10]:
#read in data sets
movies = pd.read_csv('data/ml-latest-small/movies.csv')

ratings = pd.read_csv('data/ml-latest-small/ratings.csv')
ratings_agg = ratings[['movieId','rating']].groupby('movieId',as_index=False).mean()

imdb = pd.read_csv('data/ml-latest-small/scraped_imdb_data.csv')
imdb_data = imdb[['Director','Runtime','movieId']]

In [11]:
#join tables 
movies.set_index('movieId',inplace=True)
ratings_agg.set_index('movieId',inplace=True)
imdb_data.set_index('movieId',inplace=True)
join_1 = movies.join(ratings_agg,how='inner')
join_2 = join_1.join(imdb_data,how = 'inner')
mov_data_sm = join_2

In [12]:
#preprocess genres
genres = mov_data_sm['genres'].astype(str).apply(lambda s:s.split('|'),0)
genres_bin = pd.get_dummies(genres.apply(pd.Series).stack()).sum(level=0)
mov_data_sm.drop('genres',axis=1,inplace=True)
mov_data_sm = mov_data_sm.join(genres_bin)

In [13]:
#preprocess runtime
mov_data_sm['Runtime'] = mov_data_sm['Runtime'].astype(str).apply(lambda s:s.strip(' min'),0)
def time_to_int(t):
    try:
        return int(t)
    except:
        return 90
mov_data_sm['Runtime'] = mov_data_sm['Runtime'].apply(time_to_int,0).astype(int)

In [14]:
#preprocess directors 
mov_data_sm = pd.get_dummies(mov_data_sm,columns=['Director'])

In [15]:
#drop title
mov_data_sm.drop('title',axis=1,inplace=True)

In [20]:
#read in posters and return normalized image dim of (300,450,3)
from PIL import Image
from resizeimage import resizeimage

def get_img_arr(movieid):
    img = Image.open("/Users/richardwen/Desktop/posters/poster_"+str(movieid)+".jpg")
    img = resizeimage.resize_cover(img,(150,225))
    return np.array(img)*1.0/256

In [22]:
#build input for CNN 
images = []
for i in mov_data_sm.index:
    print i
    images.append(get_img_arr(i))
images = np.array(images)
print(images.shape)

3197
3198
3199
3200
3201
3202
3203
3204
3205
3206
3207
3208
3210
3211
3213
3214
3216
3217
3218
3219
3221
3223
3224
3225
3232
3235
3238
3239
3240
3241
3243
3244
3245
3246
3247
3248
3249
3250
3251
3252
3253
3254
3255
3256
3257
3258
3259
3260
3261
3262
3263
3264
3265
3266
3267
3268
3269
3270
3271
3272
3273
3274
3275
3276
3281
3282
3284
3285
3286
3287
3289
3292
3294


ImageSizeError: 'Image is too small, Image size : (89, 137), Required size : (150, 225)'

In [ ]:
from sklearn.model_selection import train_test_split
y = mov_data_sm['rating']
x = mov_data_sm.drop('rating',axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=21)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
rfg = RandomForestRegressor(200)
rfg.fit(x_train,y_train)
pred = rfg.predict(x_test)
print mean_squared_error(pred,y_test)

In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [8]:
model = Sequential()
model.add(Conv2D(32,(3,3),input_shape = (450,300,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('linear'))

opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

model.compile(loss='mean_squared_error',optimizer=opt,metrics=['mean_squared_error'])